# DBSCAN Model Training

In [0]:
import numpy as np
import pandas as pd
import math

import json

import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.spatial.distance import cdist

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, udf
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

from sklearn.cluster import DBSCAN
from sklearn.neighbors import KNeighborsClassifier

In [0]:
# Load selected features
feature_cols_json = dbutils.fs.head("dbfs:/tmp/feature_cols.json")
feature_cols = json.loads(feature_cols_json)
feature_cols

Out[2]: ['temperature_2m',
 'precipitation',
 'wind_speed_10m',
 'wind_speed_80m',
 'wind_speed_120m',
 'wind_speed_180m',
 'cloud_cover']

In [0]:
# Load data from DBFS
sdf = spark.table("historical_weather_hourly_lisbon")
display(sdf)

temperature_2m,precipitation,wind_speed_10m,wind_speed_80m,wind_speed_120m,wind_speed_180m,cloud_cover
11.6,0.1,2.9,13.6,17.7,18.8,83.0
11.9,0.0,4.0,15.0,18.9,22.1,79.0
12.5,0.0,4.6,13.3,17.8,20.2,80.0
13.9,0.1,9.4,16.2,22.9,27.3,100.0
15.0,1.0,12.8,22.5,23.9,25.4,89.0
15.3,0.5,15.0,27.4,29.6,31.4,100.0
15.4,0.3,13.8,25.1,27.3,28.7,100.0
15.4,0.4,12.6,22.4,24.2,26.0,100.0
15.2,0.5,11.1,19.8,21.6,23.4,100.0
15.4,0.4,9.2,16.6,18.3,20.4,100.0


In [0]:
print(sdf.rdd.getNumPartitions())

sdf = sdf.repartition(2)

print(sdf.rdd.getNumPartitions())

2
2
